In [ ]:
 ! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"erlanggasatrioagung","key":"3b87a48f5cff18c3f6741632ec900bfb"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp '/content/kaggle.json' ~/.kaggle/

In [ ]:
 ! chmod 600 /content/kaggle.json

In [ ]:
! kaggle datasets download -d msambare/fer2013

 96% 58.0M/60.3M [00:03<00:00, 24.4MB/s]
100% 60.3M/60.3M [00:03<00:00, 17.8MB/s]


In [ ]:
!mkdir Dataset
!cp /content/fer2013.zip /content/Dataset


In [ ]:
!unzip -q /content/Dataset/fer2013.zip -d /content/Dataset

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (48, 48)
BATCH_SIZE = 32
train_dir = '/content/Dataset/train'
val_dir = '/content/Dataset/test'

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=img_size,
                                               class_mode='categorical',
                                               batch_size=BATCH_SIZE,
                                               color_mode='grayscale',
                                               subset='training') #setting_training_data

validation_data = validation_datagen.flow_from_directory(val_dir,
                                                         target_size=img_size,
                                                         class_mode='categorical',
                                                         batch_size=BATCH_SIZE,
                                                         color_mode='grayscale') #setting_validation_data


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, Flatten, Dense, MaxPooling2D
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape = (48, 48, 1)),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7, activation='softmax')
])

model.summary()
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy",
               tf.keras.metrics.Precision(),
               tf.keras.metrics.Recall()]
    )

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 128)        0

In [ ]:
 history = model.fit(
    train_data,
    steps_per_epoch = train_data.samples//BATCH_SIZE,
    epochs = 400,
    validation_data = validation_data,
    validation_steps = validation_data.samples//BATCH_SIZE
    )

Epoch 1/400
897/897 [==============================] - 26s 29ms/step - loss: 1.5736 - accuracy: 0.3898 - precision: 0.6678 - recall: 0.1260 - val_loss: 1.4473 - val_accuracy: 0.4436 - val_precision: 0.8404 - val_recall: 0.1403
Epoch 2/400
897/897 [==============================] - 26s 29ms/step - loss: 1.5138 - accuracy: 0.4163 - precision: 0.7015 - recall: 0.1603 - val_loss: 1.4300 - val_accuracy: 0.4436 - val_precision: 0.7658 - val_recall: 0.2080
Epoch 3/400
897/897 [==============================] - 26s 28ms/step - loss: 1.4797 - accuracy: 0.4270 - precision: 0.7081 - recall: 0.1749 - val_loss: 1.3694 - val_accuracy: 0.4771 - val_precision: 0.7903 - val_recall: 0.2125
Epoch 4/400
897/897 [==============================] - 26s 29ms/step - loss: 1.4496 - accuracy: 0.4431 - precision: 0.7140 - recall: 0.1917 - val_loss: 1.3485 - val_accuracy: 0.4859 - val_precision: 0.7938 - val_recall: 0.2160
Epoch 5/400
897/897 [==============================] - 26s 29ms/step - loss: 1.4287 - accura

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
export_dir = '/tmp/saved_model'
tf.saved_model.save(model, export_dir)